# Teste Local - Geração de Backgrounds

Este notebook testa localmente o mesmo processo que está rodando no servidor.
Vamos simular exatamente o que a API faz.

In [ ]:
# Imports exatamente iguais ao servidor
import pathlib
import base64
import requests
import tempfile
import random
import time
import openai
from openai import OpenAI
from supabase import create_client, Client
import os

print(f"🔍 Versão OpenAI: {openai.__version__}")

# Configuração igual ao servidor
IMAGES_DIR = pathlib.Path("stored_images")
BGS_DIR = pathlib.Path("bgs")
API_KEY = "a2c4457ed6a14299a425dd670e5a8ad0"

# Cliente OpenAI igual ao servidor
client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.aimlapi.com/v1",
)

print("✅ Configuração carregada")

In [ ]:
# Funções exatamente iguais ao servidor
def save_from_url(url: str, out_path: pathlib.Path):
    r = requests.get(url, stream=True)
    r.raise_for_status()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(8192):
            if chunk:
                f.write(chunk)

def find_image_by_name(image_name: str) -> pathlib.Path:
    """Encontra uma imagem pelo nome, com ou sem extensão."""
    # Se já tem extensão, usa diretamente
    if '.' in image_name:
        image_path = IMAGES_DIR / image_name
        if image_path.exists():
            return image_path
    else:
        # Busca por extensões suportadas
        extensions = ['.jpg', '.jpeg', '.png', '.webp', '.svg']
        for ext in extensions:
            image_path = IMAGES_DIR / f"{image_name}{ext}"
            if image_path.exists():
                return image_path
    
    raise Exception(f"Imagem '{image_name}' não encontrada")

print("✅ Funções carregadas")

In [ ]:
# INPUT: Nome do time (igual à API)
team_name = "flamengo"  # ⬅️ MUDE AQUI O NOME DO TIME

print(f"🏆 Time selecionado: {team_name}")

# Verificar se as pastas existem
print(f"📁 Pasta stored_images existe: {IMAGES_DIR.exists()}")
print(f"📁 Pasta bgs existe: {BGS_DIR.exists()}")

if BGS_DIR.exists():
    bg_files = [f for f in BGS_DIR.iterdir() if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.webp']]
    print(f"📸 BGs disponíveis: {len(bg_files)}")
else:
    print("❌ Pasta bgs não encontrada!")

In [ ]:
# PROCESSO PRINCIPAL: Exatamente igual ao servidor
try:
    # Encontra o escudo do time
    team_logo_path = find_image_by_name(team_name)
    print(f"🎯 Escudo encontrado: {team_logo_path}")
    
    # Pega 5 imagens aleatórias da pasta bgs
    if not BGS_DIR.exists():
        raise Exception("Pasta de backgrounds não encontrada")
    
    bg_files = [f for f in BGS_DIR.iterdir() if f.is_file() and f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.webp']]
    if len(bg_files) < 5:
        raise Exception("Não há backgrounds suficientes na pasta bgs")
    
    selected_bgs = random.sample(bg_files, 5)
    print(f"🎲 BGs selecionados: {[bg.name for bg in selected_bgs]}")
    
    # Criar pasta para salvar resultados
    results_dir = pathlib.Path(f"results_{team_name}")
    results_dir.mkdir(exist_ok=True)
    
    generated_files = []
    
    for i, bg_path in enumerate(selected_bgs):
        try:
            bg_original_name = bg_path.stem
            print(f"\n🔄 Processando {bg_original_name} ({i+1}/5)...")
            
            # Abre as imagens para a API (EXATAMENTE igual ao servidor)
            images = [open(bg_path, "rb"), open(team_logo_path, "rb")]
            
            print(f"🔍 Tipo das imagens: {type(images)}")
            print(f"🔍 Tamanho da lista: {len(images)}")
            print(f"🔍 Tipo do primeiro item: {type(images[0])}")
            print(f"🔍 Tipo do segundo item: {type(images[1])}")
            
            try:
                # Usar cliente OpenAI EXATAMENTE igual ao servidor
                result = client.images.edit(
                    model="openai/gpt-image-1",
                    image=images,
                    prompt="Quero esse fundo com a mesma forma e estilo porem com as cores desse escudo do escudo.png, coloque o logo com opacidade 50% para parecer integrado ao fundo",
                    size="1024x1536"
                )
                
                choice = result.data[0]
                print(f"✅ API respondeu com sucesso!")
                print(f"🔍 Resposta tem URL: {hasattr(choice, 'url') and choice.url is not None}")
                print(f"🔍 Resposta tem b64_json: {hasattr(choice, 'b64_json') and choice.b64_json is not None}")
                
                # Salva a imagem localmente
                output_filename = f"{team_name}_{bg_original_name}.png"
                output_path = results_dir / output_filename
                
                if getattr(choice, "url", None):
                    save_from_url(choice.url, output_path)
                    print(f"💾 Salvo via URL: {output_path}")
                elif getattr(choice, "b64_json", None):
                    img_bytes = base64.b64decode(choice.b64_json)
                    with open(output_path, "wb") as f:
                        f.write(img_bytes)
                    print(f"💾 Salvo via base64: {output_path}")
                else:
                    print("⚠️ Resposta inesperada da API")
                    continue
                    
                generated_files.append(str(output_path))
                
            finally:
                # Fechar arquivos
                for f in images:
                    f.close()
                    
        except Exception as e:
            print(f"❌ Erro ao processar {bg_path.name}: {type(e).__name__}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue
    
    print(f"\n🎉 Processo concluído!")
    print(f"📊 Imagens geradas: {len(generated_files)}")
    for i, file in enumerate(generated_files, 1):
        print(f"  {i}. {file}")
        
except Exception as e:
    print(f"❌ Erro geral: {type(e).__name__}: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
# Mostrar as imagens geradas (se houver)
from IPython.display import Image, display
import matplotlib.pyplot as plt

if 'generated_files' in locals() and generated_files:
    print(f"🖼️ Exibindo {len(generated_files)} imagens geradas:")
    
    for i, file_path in enumerate(generated_files):
        if os.path.exists(file_path):
            print(f"\n📸 Imagem {i+1}: {os.path.basename(file_path)}")
            display(Image(file_path))
        else:
            print(f"❌ Arquivo não encontrado: {file_path}")
else:
    print("❌ Nenhuma imagem foi gerada para exibir")